In [ ]:
%matplotlib inline
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile
from parcels import ErrorCode
import numpy as np
import math
from datetime import timedelta
from operator import attrgetter

In [ ]:
fieldset = FieldSet.from_parcels("MovingEddies_data/moving_eddies", allow_time_extrapolation=True)

In [ ]:
fieldset.U.show()

In [ ]:
pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=JITParticle,  # the type of particles (JITParticle or ScipyParticle)
                             lon=[3.5e5,  3.5e5], # a vector of release longitudes 
                             lat=[1e5, 3.1e5],
                             repeatdt=86400)    # a vector of release latitudes

In [ ]:
pset.show(field=fieldset.U)

In [ ]:
# Hydrodynamic drag kernel
def WaterDrag(particle, fieldset, time):
        (u, v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        Ac = 386.28 # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 48.65 # total mass of the kelp plant being simulated
        expo = 0.95
        waterdrag = ((0.5*1.025*(u*u)*0.47*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        particle.lon += (u - waterdrag) # overall velocity minus the loss in the velocity as a result of drag
        particle.lat += (v - waterdrag) # overall velocity minus the loss in the velocity as a result of drag
        
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()

In [ ]:
#kernels = pset.Kernel(AdvectionRK4) # KelpFloat
kernels = pset.Kernel(AdvectionRK4) + WaterDrag # KelpFloat

In [ ]:
#output_file = pset.ParticleFile(name="EddyParticles.nc", outputdt=timedelta(hours=1)) # the file name and the time step of the outputs
output_file = pset.ParticleFile(name="EddyKelp_Mean.nc", outputdt=timedelta(hours=1))

pset.execute(kernels,                 # the kernel (which defines how particles move)
             runtime=timedelta(days=5),    # the total length of the run
             dt=timedelta(minutes=5),      # the timestep of the kernel
             output_file=output_file,
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
pset.show(field=fieldset.U)

In [ ]:
output_file.export()
plotTrajectoriesFile('EddyParticles.nc');
#plotTrajectoriesFile('EddyKelp_Mean.nc');

In [ ]:
#plotTrajectoriesFile('EddyParticles.nc', mode='movie2d_notebook')
plotTrajectoriesFile('EddyKelp_Mean.nc', mode='movie2d_notebook')

In [ ]:
plotTrajectoriesFile('EddyParticles.nc', mode='hist2d', bins=[30, 20]);
#plotTrajectoriesFile('EddyKelp_Mean.nc', mode='hist2d', bins=[30, 20]);